# 노래가사로 장르 예측하기

In [1]:
# import
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import re # 정규식

### 데이터 불러오기

In [2]:
songs = pd.read_csv("songs_fin.csv")
songs.head()

,genre,song_id,artist_id,song_name,artist_name,lyric
0,포크,31263577,468244,매트리스,10CM,오늘밤 너는 나와 이불 속에 들어가 아무것도 하지 말고 그냥 바라보다가 웃음을 참지...
1,포크,8194007,968452,오늘,오왠 (O.WHEN),새벽4시 잠들지 않아 돌아갈 수 없는 시간들을 생각하곤 해 습관처럼 마음이 아려...
2,포크,30657311,792022,나의 사춘기에게,볼빨간사춘기,나는 한때 내가 이 세상에 사라지길 바랬어 온 세상이 너무나 캄캄해 매일 밤을 울던...
3,포크,30611680,468244,폰서트,10CM,이건 세상에서 제일 비싼 단독 공연 가수는 나고 관객은 너 하나 화려한 막이 이제 ...
4,포크,9620473,792022,나만 안되는 연애,볼빨간사춘기,왠지 오늘따라 마음이 아픈지 했더니 오늘은 그대가 날 떠나가는 날이래요 왜 항상 나...


### 장르별 샘플 수 확인

In [3]:
songs['genre'].value_counts()

발라드    559
힙합     486
댄스     448
트로트    435
포크     433
국악     418
Name: genre, dtype: int64

### 데이터 정제 None, <> , 불경 등

In [4]:
songs[songs['lyric'] == 'None']['genre'].value_counts()

국악    108
힙합     29
포크      1
Name: genre, dtype: int64

In [5]:
print(559,
486 - 29,
448,
435,
433 - 1,
418 - 108)

559 457 448 435 432 310


In [6]:
songs.drop(songs[songs['lyric'] == 'None'].index , inplace= True)
songs

,genre,song_id,artist_id,song_name,artist_name,lyric
0,포크,31263577,468244,매트리스,10CM,오늘밤 너는 나와 이불 속에 들어가 아무것도 하지 말고 그냥 바라보다가 웃음을 참지...
1,포크,8194007,968452,오늘,오왠 (O.WHEN),새벽4시 잠들지 않아 돌아갈 수 없는 시간들을 생각하곤 해 습관처럼 마음이 아려...
2,포크,30657311,792022,나의 사춘기에게,볼빨간사춘기,나는 한때 내가 이 세상에 사라지길 바랬어 온 세상이 너무나 캄캄해 매일 밤을 울던...
3,포크,30611680,468244,폰서트,10CM,이건 세상에서 제일 비싼 단독 공연 가수는 나고 관객은 너 하나 화려한 막이 이제 ...
4,포크,9620473,792022,나만 안되는 연애,볼빨간사춘기,왠지 오늘따라 마음이 아픈지 했더니 오늘은 그대가 날 떠나가는 날이래요 왜 항상 나...
...,...,...,...,...,...,...
2774,트로트,33093936,994944,별 빛이 내린다,임영웅,그 밤에 그 밤 사랑하는 사람들 품으로 그 밤에 그 밤 지나간 추억에 따스함 위로 ...
2775,트로트,33061061,994944,휘파람,임영웅,그대 떠난 여기 노을진 산마루턱엔 아직도 그대 향기가 남아서 이렇게 서있오 ...
2776,포크,5486035,467880,첫눈이 오면,스탠딩 에그,손이 시려오면 아직도 그날이 어제인 것 같아 같은 주머니에 따뜻했던 우리 두...
2777,국악,33088143,2306494,BIRD,서진실,새야새야 파랑새야 녹두밭에 앉지마라 녹두꽃이 떨어진다 청포장수 울고간다 새야...


In [7]:
songs.drop(songs[songs['lyric'].str.contains("<")].index , inplace= True)
songs

,genre,song_id,artist_id,song_name,artist_name,lyric
0,포크,31263577,468244,매트리스,10CM,오늘밤 너는 나와 이불 속에 들어가 아무것도 하지 말고 그냥 바라보다가 웃음을 참지...
1,포크,8194007,968452,오늘,오왠 (O.WHEN),새벽4시 잠들지 않아 돌아갈 수 없는 시간들을 생각하곤 해 습관처럼 마음이 아려...
2,포크,30657311,792022,나의 사춘기에게,볼빨간사춘기,나는 한때 내가 이 세상에 사라지길 바랬어 온 세상이 너무나 캄캄해 매일 밤을 울던...
3,포크,30611680,468244,폰서트,10CM,이건 세상에서 제일 비싼 단독 공연 가수는 나고 관객은 너 하나 화려한 막이 이제 ...
4,포크,9620473,792022,나만 안되는 연애,볼빨간사춘기,왠지 오늘따라 마음이 아픈지 했더니 오늘은 그대가 날 떠나가는 날이래요 왜 항상 나...
...,...,...,...,...,...,...
2774,트로트,33093936,994944,별 빛이 내린다,임영웅,그 밤에 그 밤 사랑하는 사람들 품으로 그 밤에 그 밤 지나간 추억에 따스함 위로 ...
2775,트로트,33061061,994944,휘파람,임영웅,그대 떠난 여기 노을진 산마루턱엔 아직도 그대 향기가 남아서 이렇게 서있오 ...
2776,포크,5486035,467880,첫눈이 오면,스탠딩 에그,손이 시려오면 아직도 그날이 어제인 것 같아 같은 주머니에 따뜻했던 우리 두...
2777,국악,33088143,2306494,BIRD,서진실,새야새야 파랑새야 녹두밭에 앉지마라 녹두꽃이 떨어진다 청포장수 울고간다 새야...


In [8]:
songs.drop(songs[songs['artist_name'].str.contains("스님")].index , inplace= True)
songs

,genre,song_id,artist_id,song_name,artist_name,lyric
0,포크,31263577,468244,매트리스,10CM,오늘밤 너는 나와 이불 속에 들어가 아무것도 하지 말고 그냥 바라보다가 웃음을 참지...
1,포크,8194007,968452,오늘,오왠 (O.WHEN),새벽4시 잠들지 않아 돌아갈 수 없는 시간들을 생각하곤 해 습관처럼 마음이 아려...
2,포크,30657311,792022,나의 사춘기에게,볼빨간사춘기,나는 한때 내가 이 세상에 사라지길 바랬어 온 세상이 너무나 캄캄해 매일 밤을 울던...
3,포크,30611680,468244,폰서트,10CM,이건 세상에서 제일 비싼 단독 공연 가수는 나고 관객은 너 하나 화려한 막이 이제 ...
4,포크,9620473,792022,나만 안되는 연애,볼빨간사춘기,왠지 오늘따라 마음이 아픈지 했더니 오늘은 그대가 날 떠나가는 날이래요 왜 항상 나...
...,...,...,...,...,...,...
2774,트로트,33093936,994944,별 빛이 내린다,임영웅,그 밤에 그 밤 사랑하는 사람들 품으로 그 밤에 그 밤 지나간 추억에 따스함 위로 ...
2775,트로트,33061061,994944,휘파람,임영웅,그대 떠난 여기 노을진 산마루턱엔 아직도 그대 향기가 남아서 이렇게 서있오 ...
2776,포크,5486035,467880,첫눈이 오면,스탠딩 에그,손이 시려오면 아직도 그날이 어제인 것 같아 같은 주머니에 따뜻했던 우리 두...
2777,국악,33088143,2306494,BIRD,서진실,새야새야 파랑새야 녹두밭에 앉지마라 녹두꽃이 떨어진다 청포장수 울고간다 새야...


### 장르 종류 확인

In [9]:
songs['genre'].unique()

array(['포크', '국악', '발라드', '댄스', '힙합', '트로트'], dtype=object)

### 국악 크로스오버 프로그램 수록곡 제외하기

In [10]:
songName = ['바다' , '바다 끝','가장 무도회','크게 라디오를 켜고','Run Devil Run' , '소리쳐봐','매일 매일 기다려' ,'고향생각','하얀나비','살아야지','달빛 창가에서',
    '매직 카펫 라이드','개여울','잊지 말기로 해','고래사냥','품바','리듬속에 그 춤을','화(火花)','마왕','희망의 아리랑','어느 60대 노부부 이야기','봄비','하늘을 달리다','연인','진정 난 몰랐네',
    '사랑가','우리집','강강술래','짝사랑','사이다','문을 여시오','어머니와 고등어','싸구려 커피','창밖의 여자','루씰','딩가딩가','빙빙빙','Dinosaur' ,'막걸리나',
    'Pick me', '홀로' ,'봄처녀','나팔바지','뱃노래','주저하는 연인들을 위해','나가거든','아무노래','어매'
]
len(songName)

48

In [11]:
indexnum = 0
findlist = []
for song in songName:
    indexnum = 0
    for data in songs[songs['genre']=='국악']['song_name'] :
        txt = re.match(song, data)
        if txt != None:
            findlist.append(indexnum)
            # print(txt)
        indexnum += 1


In [12]:
songs[songs['genre'] == '국악'].iloc[findlist]

,genre,song_id,artist_id,song_name,artist_name,lyric
885,국악,34445920,2898232,바다,서도밴드 (sEODo BAND),저 파도가 넘실거린다 쌩쌩 부는 저 바람에 실려 어디서부터 오는 걸까 끊임없이 또 ...
894,국악,34408475,2898232,바다 끝,서도밴드 (sEODo BAND),먼 아주 멀리 있는 저 바다 끝보다 까마득한 그곳에 태양처럼 뜨겁던 내 사랑을...
1914,국악,33834600,2862208,바다에 누워,정초롱,어기야 차 어기야 차 어기야 차 어기야 아아 어기야 어기야 차 어기야 차차 저 바다...
894,국악,34408475,2898232,바다 끝,서도밴드 (sEODo BAND),먼 아주 멀리 있는 저 바다 끝보다 까마득한 그곳에 태양처럼 뜨겁던 내 사랑을...
935,국악,34408476,2945321,가장 무도회,모던판소리공작소 촘촘,해가 지면 거리는 잿빛 화장하고 언제나 표정 없는 얼굴로 사랑하지 않아도 애인 될 ...
...,...,...,...,...,...,...
908,국악,33600687,2898232,뱃노래,서도밴드 (sEODo BAND),달은 밝고 명랑한데 고향 생각 절로 난다 절로 난다 달은 밝고 명랑한데 고향 생각 ...
907,국악,34046622,2969601,주저하는 연인들을 위해,해음,나는 읽기 쉬운 마음이야 당신도 스윽 훑고 가셔요 달랠 길 없는 외로운 마음 있지 ...
903,국악,34046623,2401057,나가거든,김준수,쓸쓸한 달빛 아래 내 그림자 하나 생기거든 그땐 말해볼까요 이 마음 들어나 주라고 ...
910,국악,34046624,2633154,아무노래,음유사인,왜들 그리 다운돼있어 뭐가 문제야 something 분위기가 겁나 싸해 요새는 이런...


### 국악 중복 노래가 너무 많다! 장르 포기하기ㅜ

In [13]:
songs.drop(songs[(songs['genre'] == '포크') | (songs['genre'] == '국악')].index , inplace= True)
songs.head()

,genre,song_id,artist_id,song_name,artist_name,lyric
461,발라드,34431086,839736,취중고백,김민석 (멜로망스),뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...
462,발라드,34481682,261143,겨울잠,아이유,때 이른 봄 몇 송이 꺾어다 너의 방 문 앞에 두었어 긴 잠 실컷 자고 나오면 그때...
463,발라드,34061322,994944,사랑은 늘 도망가,임영웅,눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...
464,발라드,34360855,2138620,눈이 오잖아(Feat.헤이즈),이무진,한 달 좀 덜 된 기억들 주머니에 넣은 채 걷고 있어 몇 시간을 혹시 몰라 네가 좋...
465,발라드,33496587,2863470,다정히 내 이름을 부르면,경서예지,끝없이 별빛이 내리던 밤 기분 좋은 바람이 두 빰을 스치고 새벽 바다 한곳을 보는 ...


In [14]:
songs['genre'].value_counts()

발라드    559
힙합     456
댄스     447
트로트    435
Name: genre, dtype: int64

In [15]:
songs[songs['lyric'].str.contains(r"]")]

,genre,song_id,artist_id,song_name,artist_name,lyric
2217,힙합,31626257,585492,IMJMWDP (Prod. By 기리보이),기리보이,[Kid Milli] IM A KING 이젠 재산도 걔네급 WHAT UH 급식의 신...


In [16]:
songs[songs['lyric'].str.contains("[()]")]
# 특수문자가 많다 

,genre,song_id,artist_id,song_name,artist_name,lyric
541,발라드,4352438,724619,첫 눈,EXO,첫눈 오는 이런 오후에 너에게 전화를 걸 수만 있다면 기쁠텐데 벌써 일년이 지났는데...
561,댄스,34349913,3055146,ELEVEN,IVE (아이브),따분한 나의 눈빛이 무표정했던 얼굴이 널 보며 빛나고 있어 널 담은 눈동자는 odd...
562,댄스,34436674,2899555,Dreams Come True,aespa,"Uh, You wanna feel the vibe That I’m feeling a..."
564,댄스,34041584,2899555,Savage,aespa,Oh my gosh! Don't you know I’m a Savage? I’m a...
565,댄스,33487342,2899555,Next Level,aespa,I’m on the Next Level Yeah 절대적 룰을 지켜 내 손을 놓지 말...
...,...,...,...,...,...,...
2707,힙합,32979568,960278,Volcano,NCT U,yeah ey 시작해 어서 몰입해 집중해 좀 더 뜨겁게 back swing부터 이른...
2735,댄스,33024054,2137482,MORE,(여자)아이들,K/DA Should we show 'em how we do it everyday ...
2738,댄스,33048921,629371,이데아 (IDEA:理想),태민 (TAEMIN),이성이 만든 작은 투영 본능을 타고 자라 깊은 곳에 핀 꿈이여 밤을 안은 채로 짙어...
2744,힙합,33029110,742773,Ok man (Feat. BOBBY),MINO (송민호),Oh ok How you like that 내 기분은 캡 When I 빡랩 교포 S...


In [17]:
songs[songs['lyric'].str.contains(r"개리")]

,genre,song_id,artist_id,song_name,artist_name,lyric
742,힙합,1637914,103289,Ballerino,리쌍,song) ali 사랑은 언제나 눈물이 돼 가슴에 남아 떠나지도 못한채 또 길을...


In [18]:
songs[songs['song_name'].str.contains("嗜")]

,genre,song_id,artist_id,song_name,artist_name,lyric
2154,댄스,32217930,724619,嗜 (Obsession),EXO,jiang zhe xun huan zhong zhi hei an jiang lin...


In [19]:
songs[songs['lyric'].str.contains("amp;")]

,genre,song_id,artist_id,song_name,artist_name,lyric
583,댄스,34491913,3058246,WA DA DA,Kep1er (케플러),Hey It’s you &amp; I Let’s start N N Now 도착한 이...
587,댄스,33999025,2622030,LOCO,ITZY (있지),LOCO 미친다는 말이 이해 간달까 I'm gettin' LOCO LOCO Oh g...
592,댄스,34538515,2399724,SMILEY (Feat. BIBI),YENA (최예나),울지 마 울지 마 어린아이같이 웃는 게 웃는 게 이기는 거라고 You are so...
598,댄스,32273582,780066,Psycho,Red Velvet (레드벨벳),Psycho 널 어쩌면 좋을까 이런 맘은 또 첨이라 Up &amp; Down이 좀...
602,댄스,32445339,2622030,WANNABE,ITZY (있지),잔소리는 Stop it 알아서 할게 내가 뭐가 되든 내가 알아서 할 테니까 좀 I ...
636,댄스,32698765,261143,Into the I-LAND,아이유,I got pushed 떠밀려 왔어 오래도 찾아 헤맨 꿈의 문 앞에 내 안에 그 작...
645,댄스,32833393,2137482,덤디덤디 (DUMDi DUMDi),(여자)아이들,뜨거운 태양에 살짝 미친 난 쉽게 두 볼이 빨개지고 그러다 어질어질 달아오른 난 ...
658,댄스,31403163,905701,YES or YES,TWICE (트와이스),Hey boy Look I’m gonna make this simple for ...
672,힙합,34331510,2758756,MBTI (Feat. 쿠기 & 로꼬) (Prod. GRAY),BE'O (비오),Million 내 몸값을 더 올려 Billion 내 몸값을 더 올려 Trillion...
684,힙합,34501948,2622202,프리지아,래원 (Layone),네가 빤히 바라봤을 때 외면하고픈 건 아니었다 이 마음이 어렵지만 관이 너무 어둡지...


] , 개리 , "&amp0;", 嗜 (Obsession) 삭제하기

In [20]:
songs['lyric'] =  songs['lyric'].str.replace('amp;' , ' ')

In [21]:
songs[songs['lyric'].str.contains("amp;")]

,genre,song_id,artist_id,song_name,artist_name,lyric


In [22]:
songs.drop(songs[songs['lyric'].str.contains(r"]")].index , inplace= True)
songs.drop(songs[songs['lyric'].str.contains(r"개리")].index , inplace= True)
songs.drop(songs[songs['song_name'].str.contains("嗜")].index , inplace= True)

### 불용어 처리하기 !!! (),/ 정규식 처리하기!

가사 정제 함수 만들기

In [23]:
def preprocess(text):
  # 한글, 영어, 숫자만 남기기
  text = re.sub(r'[^a-zA-Z0-9가-힣]', ' ', text)
  # 영문을 소문자로
  text = text.lower()
  # 두 칸 이상 공백을 한 칸으로
  text = re.sub(r'\s+', ' ', text)
  return text

In [24]:
songs['lyric'] = songs['lyric'].apply(preprocess)
songs.head()

,genre,song_id,artist_id,song_name,artist_name,lyric
461,발라드,34431086,839736,취중고백,김민석 (멜로망스),뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...
462,발라드,34481682,261143,겨울잠,아이유,때 이른 봄 몇 송이 꺾어다 너의 방 문 앞에 두었어 긴 잠 실컷 자고 나오면 그때...
463,발라드,34061322,994944,사랑은 늘 도망가,임영웅,눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...
464,발라드,34360855,2138620,눈이 오잖아(Feat.헤이즈),이무진,한 달 좀 덜 된 기억들 주머니에 넣은 채 걷고 있어 몇 시간을 혹시 몰라 네가 좋...
465,발라드,33496587,2863470,다정히 내 이름을 부르면,경서예지,끝없이 별빛이 내리던 밤 기분 좋은 바람이 두 빰을 스치고 새벽 바다 한곳을 보는 ...


In [25]:
songs['genre'].value_counts()

발라드    559
힙합     454
댄스     446
트로트    435
Name: genre, dtype: int64

In [33]:
songs.reset_index(drop='index' , inplace=True)

### 데이터 백업

In [34]:
songs = songs[['genre','lyric']]
songs.to_csv('songs_preprocessed.csv', index=False)

데이터 정제 끝

---

### 데이터에 사용된 중복 없는 전체 단어 갯수 파악 

In [28]:
result = set()
songs['lyric'].str.lower().str.split().apply(result.update)
vocab_size = len(result)
vocab_size

49244

### 단어를 숫자로 인코딩 하기

In [29]:
lyrics = songs['lyric'].to_list() # 길이가 다른 건 numpy를 못쓴다.
lyrics[0:3]

['뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들 만나서 오랜만에 술을 좀 했는데 자꾸만 니 얼굴 떠올라 무작정 달려왔어 이 맘 모르겠니 요즘 난 미친 사람처럼 너만 생각해 대책없이 네가 점점 좋아져 아냐 안 취했어 진짜야 널 정말 사랑해 눈물이 날만큼 원하고 있어 정말로 몰랐니 가끔 전화해 장난치듯 주말엔 뭐할거냐며 너의 관심 끌던 나를 그리고 한번씩 누나 주려 샀는데 너 그냥 준다고 생색 낸 선물도 너 때문에 산거야 이 맘 모르겠니 요즘 난 미친 사람처럼 너만 생각해 대책없이 네가 점점 좋아져 아냐 안 취했어 진짜야 널 정말 사랑해 진심이야 믿어줘 갑자기 이런 말 놀랐다면 미안해 부담이 되는게 당연해 이해해 널 하지만 내 고백도 이해해 주겠니 oh 지금 당장 대답하진마 나와 일주일만 사귀어줄래 후회없이 잘 해주고 싶은데 그 후에도 니가 싫다면 나 그때 포기할게 귀찮게 안할게 혼자 아플게 진심이야 너를 사랑하고 있어 ',
 '때 이른 봄 몇 송이 꺾어다 너의 방 문 앞에 두었어 긴 잠 실컷 자고 나오면 그때쯤엔 예쁘게 피어 있겠다 별 띄운 여름 한 컵 따라다 너의 머리맡에 두었어 금세 다 녹아버릴 텐데 너는 아직 혼자 쉬고 싶은가 봐 너 없이 보는 첫 봄이 여름이 괜히 왜 이렇게 예쁘니 다 가기 전에 널 보여줘야 하는데 음 꼭 봐야 하는데 내게 기대어 조각잠을 자던 그 모습 그대로 잠들었구나 무슨 꿈을 꾸니 깨어나면 이야기해 줄 거지 언제나의 아침처럼 음 빼곡한 가을 한 장 접어다 너의 우체통에 넣었어 가장 좋았던 문장 아래 밑줄 그어 나 만나면 읽어줄래 새하얀 겨울 한 숨 속에다 나의 혼잣말을 담았어 줄곧 잘 참아내다가도 가끔은 철없이 보고 싶어 새삼 차가운 연말의 공기가 뼈 틈 사이사이 시려와 움츠려 있을 너의 그 마른 어깨를 꼭 안아줘야 하는데 내게 기대어 조각잠을 자던 그 모습 그대로 잠들었구나 무슨 꿈을 꾸니 깨어나면 이야기해 줄 거지 언제나의 아침처럼 음',
 '눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸

In [35]:
songs

,genre,lyric
0,발라드,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...
1,발라드,때 이른 봄 몇 송이 꺾어다 너의 방 문 앞에 두었어 긴 잠 실컷 자고 나오면 그때...
2,발라드,눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...
3,발라드,한 달 좀 덜 된 기억들 주머니에 넣은 채 걷고 있어 몇 시간을 혹시 몰라 네가 좋...
4,발라드,끝없이 별빛이 내리던 밤 기분 좋은 바람이 두 빰을 스치고 새벽 바다 한곳을 보는 ...
...,...,...
1889,힙합,내 결 아무리 봐도 난 비단 왜 차이가 날까 너의 시간과 다른 나의 시간 이 밤 지...
1890,힙합,하고픈 말이 없어 멍하니 생각이 멈춰 고장 난 듯이 무기력해 두 발이 무거워 가자 ...
1891,힙합,baby 옷 입는 것 봐 처음이야 밥 해준 여자 놀라운 유머감각 피식해 야한 농담에...
1892,트로트,그 밤에 그 밤 사랑하는 사람들 품으로 그 밤에 그 밤 지나간 추억에 따스함 위로 ...
